In [96]:
from sentence_transformers import SentenceTransformer
import torch
import spacy
import pandas as pd
import numpy as np
import concurrent.futures
import pickle
import faiss
from transformers import pipeline
from scipy.stats import skew


pd.set_option('display.max_colwidth', 100)
nlp = spacy.load("es_core_news_sm")
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(torch.cuda.current_device()))

NVIDIA GeForce RTX 2070 SUPER


In [ ]:
model_id = "meta-llama/Llama-3.2-1B-Instruct"
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    
)

messages = [
    {"role": "system", "content": "Eres una máquina que reformula consultas sobre historia latinoamericana. A cada consulta responderás con un pasaje de texto corto pero relevante que pueda ser útil para responder a la consulta original. No vas a dar respuestas que incluyan títulos, listas o formato Markdown."},
    {"role": "user", "content": "Influencias de Colón en la historia de América"},
]
messages2 = [
    {"role": "system", "content": "Eres una máquina especializada en reformular consultas sobre historia latinoamericana. A cada consulta, tu tarea es hacerla más precisa y clara, sin extenderte demasiado. Solo responderás con la consulta reformulada, sin incluir títulos, listas o formato Markdown."},
    {"role": "user", "content": "Influencias de Colón en la historia de América"},
]
messages3 = [
    {"role": "system", "content": "Eres un experto en historia latinoamericana. Tu tarea es tomar consultas relacionadas con la historia de América Latina y reformularlas para hacerlas más precisas, claras y detalladas. Deberías asegurarte de que la nueva consulta sea más específica, completa y comprensible. Responde solo con la consulta reformulada, sin agregar respuestas completas ni contenido adicional. No uses listas, títulos ni formato Markdown."},
    {"role": "user", "content": "Influencias de Colón en la historia de América"},
]

def get_messages(q):
    return [
        {"role": "system", "content": "Eres un experto en historia latinoamericana. Tu tarea es tomar consultas relacionadas con la historia de América Latina y reformularlas para hacerlas más precisas, claras y detalladas. Deberías asegurarte de que la nueva consulta sea más específica, completa y comprensible. Responde solo con la consulta reformulada, sin agregar respuestas completas ni contenido adicional. No uses listas, títulos ni formato Markdown."},
        {"role": "user", "content": q},
    ]

Device set to use cuda:0


In [5]:
q = "Monopolio de Sevilla"

outputs = pipe(
    get_messages(q),
    max_new_tokens=256,
)

new_query = outputs[0]["generated_text"][-1]['content']
new_query

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


'El monopolio de Sevilla se refiere a la situación en la que la ciudad de Sevilla, en España, se convirtió en un centro de comercio y comerciante de bienes de lujo, especialmente vino, en el siglo XVII. Esta situación se debió a una combinación de factores, incluyendo la expansión del comercio español en América, la creciente demanda de vino en Europa y la posición geográfica de Sevilla como puerto.\n\nEn 1635, la ciudad de Sevilla se convirtió en el principal centro de comercio de vino en Europa, gracias en parte a la expansión de la industria vinícola en América. La ciudad era un importante centro de exportación de vino, y su reputación como una de las mejores tiendas de vino del mundo se consolidó en ese momento.\n\nEl monopolio de Sevilla se debió en parte a la posición de la ciudad como puerto y el centro de comercio, que la convirtió en un punto de partida para la exportación de vino a todo el mundo. Además, la ciudad tenía una gran cantidad de comerciantes y vendedores de bienes

In [ ]:
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [40]:
def divide_document(text, max_words=300):
    doc = nlp(text)
    sents = [sent.text for sent in doc.sents]
    fragments = []
    temp = ""

    for sent in sents:
        if len(temp.split()) + len(sent.split()) <= max_words:
            temp += " " + sent
        else:
            fragments.append(temp.strip())
            temp = sent

    if temp:
        fragments.append(temp.strip())

    return fragments


In [7]:
df = pd.read_csv("preprocessed.csv")
df

,Path,Text
0,pack\AIBR\2006-Aymaras-peruanos-y-chilenos-en-los-Andes-ariqueños-Resistencia-y-conflicto-frente...,"catalogada y caricaturizada bajo el rótulo de indómito pueblo araucano. esta imagen, construida ..."
1,pack\AIBR\2006-Cambios-de-género-y-discriminación-laboral-en-el-sector-financiero-colombiano-El-...,agradecimientos la autora desea agradecer a la profesora luz gabriela arango la lectura del pres...
2,pack\AIBR\2006-Consecuencias-personales-en-la-ruptura-de-la-vida-laboral-El-caso-de-Telefónica.pdf,"has carried out the biggest employment adjustment in our country, that will finished in 2007, af..."
3,pack\AIBR\2006-Construcción-de-modelos-de-género-a-partir-de-textos-de-la-tradición-oral-en-Extr...,"tampoco queremos que se entienda identidad de género, como identidad del nosotras o identidad de..."
4,pack\AIBR\2006-Contribuciones-feministas-a-problemas-epistemológicos-de-la-disciplina-antropológ...,is irrelevant for others con el que teresa del valle enuncia uno de los apartados de su introduc...
...,...,...
1920,pack\Revista-Española-de-Antropología-Americana\2024-Paridad-y-violencia-política-contra-las-muj...,"cómo citar: burguete cal y mayor, araceli. 2024. paridad y violencia política contra las mujeres..."
1921,pack\Revista-Española-de-Antropología-Americana\2024-Reorientación-de-la-educación-superior-inte...,project exposed some critical questionings. in this paper i expose five of these critical questi...
1922,pack\Revista-Española-de-Antropología-Americana\2024-Una-aproximación-metodológica-a-la-lectura-...,"1. introducción el interés por los quipus, que ha crecido en los últimos años, ha sido acompañad..."
1923,pack\Revista-Española-de-Antropología-Americana\2024-Valdivia-fase-3-transformación-de-las-figur...,1. introducción ¿cómo se produce la diferenciación entre los géneros? ¿cómo surge el conflicto d...


In [40]:
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer

# Carga el modelo y el tokenizer
model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
model = SentenceTransformer(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [43]:
mi, ma = 1 << 31, 0
for i, r in df.iterrows():
    if type(r["Text"]) != str:
        continue
    n_tokens = len(tokenizer.tokenize(r["Text"]))
    mi = min(mi, n_tokens)
    ma = max(ma, n_tokens)
mi, ma

Token indices sequence length is longer than the specified maximum sequence length for this model (8924 > 512). Running this sequence through the model will result in indexing errors


(65, 44391)

In [42]:
def divide_and_assign_ids(i, text):
    if type(text) != str:
        return [], []

    frags = divide_document(text)
    return frags, [i] * len(frags)

fragments = []
fragments_ids = []

with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    results = list(executor.map(lambda row: divide_and_assign_ids(row[0], row[1]["Text"]),
                                df.iterrows()))

    for frags, ids in results:
        if not frags or not ids:
            continue

        fragments.extend(frags)
        fragments_ids.extend(ids)

len(fragments), len(fragments_ids)

KeyboardInterrupt: 

In [ ]:
with open("fragments.pkl", "wb") as f:
    pickle.dump((fragments, fragments_ids), f)

In [28]:
with open("fragments.pkl", "rb") as f:
    fragments, fragments_ids = pickle.load(f)

In [29]:
embeddings = model.encode(fragments, convert_to_tensor=True, device=device)
print(embeddings.shape)

torch.Size([57716, 384])


In [81]:
embeddings_np = embeddings.cpu().numpy()
print(embeddings_np.shape)

# embeddings_normalized = embeddings_np / np.linalg.norm(embeddings_np, axis=1, keepdims=True)
# print(embeddings_normalized.shape)

(73111, 1024)


In [82]:
index = faiss.IndexFlatL2(embeddings_np.shape[1])
index.add(embeddings_np)
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x000001E5071ED4D0> >

In [21]:
faiss.write_index(index, "index-sentence-transformers.faiss")

In [9]:
index = faiss.read_index("index-sentence-transformers.faiss")

In [83]:
def clean_query(query: str) -> str:
    query = query.replace('"', '').replace("'", '')
    query = query.replace("\n", " ").replace("\r", " ")
    query = query.replace("  ", " ").strip()

    doc = nlp(query)
    sentences = [sent.text.strip() for sent in doc.sents]
    return " ".join([sent for sent in sentences if len(sent) > 0 and sent[-1] == '.']).lower()

def search(query, top_k=10):
    query_embedding = model.encode([clean_query(query)], convert_to_tensor=True, device=device).cpu().numpy()
    distances, indices = index.search(query_embedding, k=top_k*4)

    already_in = set()
    results = []
    for i in indices[0]:
        id = fragments_ids[i]
        if id in already_in:
            continue

        already_in.add(id)
        results.append(id)
        if len(results) == top_k:
            break

    return df.iloc[results]

def skewness(query, top_k=10):
    query_embedding = model.encode([clean_query(query)], convert_to_tensor=True, device=device).cpu().numpy()
    distances, _ = index.search(query_embedding, k=top_k)

    # max and min distances
    print(distances[0].min(), distances[0].max())

    similarities = 1 - distances
    return skew(similarities[0])

In [84]:
search(new_query, top_k=10)

,Path,Text
675,pack\Anuario-de-Estudios-Americanos\2014-La-elaboración-de-la-Ordenanza-de-Montes-de-Marina-de-3...,introducción el mayor punto del comercio de españa depende de arreglar lo casi perdido con sus i...
1736,pack\Revista-Española-de-Antropología-Americana\2015-Interacciones-entre-las-sociedades-y-las-pl...,figura 1: ubicación del área septentrional del río de la plata en el contexto regional. en detal...
1307,pack\Revista-de-Indias\2017-Gamonales-y-alcaldes-poder-institucional-y-parainstitucional-en-la-P...,introducción la presente investigación tiene como principal objetivo recalcar el importante pape...
956,pack\Revista-de-Indias\2002-En-búsqueda-de-un-punto-fijo-para-la-República-El-cesarismo-liberal-...,"and covenants, without the sword, are but words2 tres de las cuatro revoluciones de principios d..."
1116,pack\Revista-de-Indias\2008-Políticas-y-prácticas-migratorias-los-flujos-de-gallegos-y-asturiano...,"ricano de las corrientes del noroeste hispánico, luego del cubano, que mantuvo una notoria y ant..."
435,pack\Anuario-de-Estudios-Americanos\2001-El-oro-de-América-La-contribución-de-los-emigrantes-del...,"ca, y en particular el lerrouxismo. la república del emigrante describe en detalle la cultura po..."
1256,pack\Revista-de-Indias\2014-México-y-las-instituciones-republicanas-en-el-exilio-del-apoyo-del-C...,"normalizaron, el inicio de la guerra civil puso de manifiesto el apoyo inequívoco del gobierno m..."
681,pack\Anuario-de-Estudios-Americanos\2014-Los-arbitristas-y-la-América-portuguesa-15901640.pdf,"introducción a finales de diciembre de 1640, los consejeros de estado de felipe iv se reunieron ..."
1104,pack\Revista-de-Indias\2008-Construyendo-identidades-desde-la-excepcionalidad-mujer-divorciada-y...,"latinoamérica. pero en esta ocasión el nombre es el de maría baldomera fuentes segura, la protag..."
1811,pack\Revista-Española-de-Antropología-Americana\2019-Explorando-las-órbitas-tecnológicas-de-dos-...,"1. introducción treinta y cinco años después del inicio del proyecto nacional tikal, el cual des..."


In [85]:
search(q, top_k=10)

,Path,Text
609,pack\Anuario-de-Estudios-Americanos\2010-Valoración-partición-y-distribución-de-la-tierra-de-los...,"introducción en la nueva granada, a cuya jurisdicción estuvo adscrita mérida hasta 1777, se deno..."
607,pack\Anuario-de-Estudios-Americanos\2010-Sólo-una-virreina-consorte-de-la-Nueva-España-16601664-...,"en la historia colonial del nuevo mundo, el más alto poder residía en los virreyes y las activid..."
862,pack\Anuario-de-Estudios-Americanos\2022-Una-mirada-familiar-a-fray-Juan-de-Zumárraga-ocho-docum...,"en el curso de mis investigaciones en los protocolos de sevilla me he encontrado, no sin cierta ..."
1055,pack\Revista-de-Indias\2006-Empresas-instituciones-y-red-social-la-Compañía-Hispanoamericana-de-...,nos envían a sus familias y comunidades de origen y que son denominados remesas. los retornos de...
1485,pack\Revista-Española-de-Antropología-Americana\2003-Diseño-arquitectónico-patrones-de-ocupación...,1. el sitio de pachacamac pachacamac se encuentra sobre la margen derecha del río lurín y cerca ...
900,pack\Revista-de-Indias\2000-El-crecimiento-económico-de-Cuba-republicana-19021959-Una-revisión-y...,"medida perfecta y tampoco la única posible. sin embargo, su cálculo y el de los demás grandes ag..."
543,pack\Anuario-de-Estudios-Americanos\2007-Militarización-e-identidades-políticas-en-la-revolución...,"ñoles americanos a partir de 1808, este trabajo propone explorar estos procesos de construcción ..."
1314,pack\Revista-de-Indias\2017-La-política-desde-el-campo-iniciativas-locales-y-gobierno-rural-en-t...,planteo introductorio uno de los grandes atractivos de la historia política colonial del siglo x...
478,pack\Anuario-de-Estudios-Americanos\2003-Una-antinomia-protorrenacentista-secreto-de-estado-y-di...,"difícil coexistencia de ambos parámetros, el mayor énfasis de uno u otro —según los intereses es..."
541,pack\Anuario-de-Estudios-Americanos\2007-Las-rutas-del-libro-atlántico-libros-enviados-en-el-nav...,to impreso fue un producto cultural con rasgos novedosos que pronto encontraría su lugar en el m...


In [27]:
skewness(q, top_k=20)

11.059961 14.668214


3.362548656394798

# LANGCHAIN

In [55]:
from sentence_transformers import SentenceTransformer
from langchain.vectorstores import FAISS
import faiss
import torch
from langchain.text_splitter import CharacterTextSplitter
import spacy
import pandas as pd
import numpy as np
import concurrent.futures
import pickle
from transformers import pipeline
from scipy.stats import skew
from langchain_text_splitters import TokenTextSplitter


pd.set_option('display.max_colwidth', 100)
nlp = spacy.load("es_core_news_sm")
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(torch.cuda.current_device()))

NVIDIA GeForce RTX 2070 SUPER


In [62]:
df = pd.read_csv("preprocessed.csv")
df

,Path,Text
0,pack\AIBR\2006-Aymaras-peruanos-y-chilenos-en-los-Andes-ariqueños-Resistencia-y-conflicto-frente...,"catalogada y caricaturizada bajo el rótulo de indómito pueblo araucano. esta imagen, construida ..."
1,pack\AIBR\2006-Cambios-de-género-y-discriminación-laboral-en-el-sector-financiero-colombiano-El-...,agradecimientos la autora desea agradecer a la profesora luz gabriela arango la lectura del pres...
2,pack\AIBR\2006-Consecuencias-personales-en-la-ruptura-de-la-vida-laboral-El-caso-de-Telefónica.pdf,"has carried out the biggest employment adjustment in our country, that will finished in 2007, af..."
3,pack\AIBR\2006-Construcción-de-modelos-de-género-a-partir-de-textos-de-la-tradición-oral-en-Extr...,"tampoco queremos que se entienda identidad de género, como identidad del nosotras o identidad de..."
4,pack\AIBR\2006-Contribuciones-feministas-a-problemas-epistemológicos-de-la-disciplina-antropológ...,is irrelevant for others con el que teresa del valle enuncia uno de los apartados de su introduc...
...,...,...
1920,pack\Revista-Española-de-Antropología-Americana\2024-Paridad-y-violencia-política-contra-las-muj...,"cómo citar: burguete cal y mayor, araceli. 2024. paridad y violencia política contra las mujeres..."
1921,pack\Revista-Española-de-Antropología-Americana\2024-Reorientación-de-la-educación-superior-inte...,project exposed some critical questionings. in this paper i expose five of these critical questi...
1922,pack\Revista-Española-de-Antropología-Americana\2024-Una-aproximación-metodológica-a-la-lectura-...,"1. introducción el interés por los quipus, que ha crecido en los últimos años, ha sido acompañad..."
1923,pack\Revista-Española-de-Antropología-Americana\2024-Valdivia-fase-3-transformación-de-las-figur...,1. introducción ¿cómo se produce la diferenciación entre los géneros? ¿cómo surge el conflicto d...


In [86]:
# model = SentenceTransformer("sentence-transformers/static-similarity-mrl-multilingual-v1", trust_remote_code=True)
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2", trust_remote_code=True)

In [90]:
text_splitter = TokenTextSplitter(chunk_size=300, chunk_overlap=50)

In [91]:
chunks, chunks_ids = [], []
for i, r in df.iterrows():
    if type(r["Text"]) != str:
        continue

    new_chunks = text_splitter.split_text(r["Text"])
    chunks.extend(new_chunks)
    chunks_ids.extend([i] * len(new_chunks))

len(chunks)

134295

In [94]:
# embeddings = model.encode(chunks, convert_to_tensor=True, device=device, prompt_name="passage")
embeddings = model.encode(chunks, convert_to_tensor=True, device=device)
embeddings.shape

KeyboardInterrupt: 

In [95]:
embeddings_np = embeddings.cpu().numpy()
embeddings_np.shape

(100, 384)

In [ ]:
index = faiss.IndexFlatL2(embeddings_np.shape[1])
index.add(embeddings_np)

faiss.write_index(index, "index-nomic-ai.faiss")

faiss_store = FAISS(index)

# Cargar embeddings y LLaMa

In [81]:
from sentence_transformers import SentenceTransformer
from langchain.vectorstores import FAISS
import faiss
import torch
from langchain.text_splitter import CharacterTextSplitter
import spacy
import pandas as pd
import numpy as np
import concurrent.futures
import pickle
from transformers import pipeline
from scipy.stats import skew
from langchain_text_splitters import TokenTextSplitter
from huggingface_hub import InferenceClient
import os
from dotenv import load_dotenv

load_dotenv()

pd.set_option('display.max_colwidth', 100)
nlp = spacy.load("es_core_news_sm")
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(torch.cuda.current_device()))

NVIDIA GeForce RTX 2070 SUPER


In [2]:
df = pd.read_csv("preprocessed.csv")
df

,Path,Text
0,pack\AIBR\2006-Aymaras-peruanos-y-chilenos-en-los-Andes-ariqueños-Resistencia-y-conflicto-frente...,"catalogada y caricaturizada bajo el rótulo de indómito pueblo araucano. esta imagen, construida ..."
1,pack\AIBR\2006-Cambios-de-género-y-discriminación-laboral-en-el-sector-financiero-colombiano-El-...,agradecimientos la autora desea agradecer a la profesora luz gabriela arango la lectura del pres...
2,pack\AIBR\2006-Consecuencias-personales-en-la-ruptura-de-la-vida-laboral-El-caso-de-Telefónica.pdf,"has carried out the biggest employment adjustment in our country, that will finished in 2007, af..."
3,pack\AIBR\2006-Construcción-de-modelos-de-género-a-partir-de-textos-de-la-tradición-oral-en-Extr...,"tampoco queremos que se entienda identidad de género, como identidad del nosotras o identidad de..."
4,pack\AIBR\2006-Contribuciones-feministas-a-problemas-epistemológicos-de-la-disciplina-antropológ...,is irrelevant for others con el que teresa del valle enuncia uno de los apartados de su introduc...
...,...,...
1920,pack\Revista-Española-de-Antropología-Americana\2024-Paridad-y-violencia-política-contra-las-muj...,"cómo citar: burguete cal y mayor, araceli. 2024. paridad y violencia política contra las mujeres..."
1921,pack\Revista-Española-de-Antropología-Americana\2024-Reorientación-de-la-educación-superior-inte...,project exposed some critical questionings. in this paper i expose five of these critical questi...
1922,pack\Revista-Española-de-Antropología-Americana\2024-Una-aproximación-metodológica-a-la-lectura-...,"1. introducción el interés por los quipus, que ha crecido en los últimos años, ha sido acompañad..."
1923,pack\Revista-Española-de-Antropología-Americana\2024-Valdivia-fase-3-transformación-de-las-figur...,1. introducción ¿cómo se produce la diferenciación entre los géneros? ¿cómo surge el conflicto d...


In [ ]:
# Para correr llama localmente

model_id = "meta-llama/Llama-3.2-1B-Instruct"
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

def get_messages(q):
    return [
        {"role": "system", "content": "Eres un experto en historia latinoamericana. Tu tarea es tomar consultas relacionadas con la historia de América Latina y reformularlas para hacerlas más precisas, claras y detalladas. Deberías asegurarte de que la nueva consulta sea más específica, completa y comprensible. Responde solo con la consulta reformulada, sin agregar respuestas completas ni contenido adicional. No uses listas, títulos ni formato Markdown."},
        {"role": "user", "content": q},
    ]

def rewrite(q: str) -> str:
    outputs = pipe(
        get_messages(q),
        max_new_tokens=256,
    )

    return outputs[0]["generated_text"][-1]['content']

Device set to use cuda:0


In [82]:

client = InferenceClient(
	provider="sambanova",
	api_key=os.getenv("HF_TOKEN")
)

def rewrite(q: str) -> str:
    messages = [
        {
            "role": "system",
            "content": "Eres un experto en historia latinoamericana. Tu tarea es tomar consultas relacionadas con la historia de América Latina y reformularlas para hacerlas más precisas, claras y detalladas. Deberías asegurarte de que la nueva consulta sea más específica, completa y comprensible. Responde solo con la consulta reformulada, sin agregar respuestas completas ni contenido adicional. No uses listas, títulos ni formato Markdown."
        },
        {"role": "user", "content": q}
    ]

    completion = client.chat.completions.create(
        model="meta-llama/Llama-3.2-1B-Instruct", 
        messages=messages, 
        max_tokens=256,
    )

    return completion.choices[0].message.content

In [13]:
model = SentenceTransformer("nomic-ai/nomic-embed-text-v2-moe", trust_remote_code=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [5]:
index = faiss.read_index("index-chunks-512-50-nomic.faiss")
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x00000187F0FF4150> >

In [6]:
embeddings_np = np.load("embeddings-chunks-512-50-nomic.bin.npy")
embeddings_np.shape

(73111, 768)

In [7]:
with open("chunks-512-50-nomic.pkl", "rb") as f:
    chunks, chunks_ids = pickle.load(f)

In [67]:
def clean_query(query: str) -> str:
    query = query.replace('"', '').replace("'", '')
    query = query.replace("\n", " ").replace("\r", " ")
    query = query.replace("*", "").replace("-", "").replace("~", "")
    query = query.replace("  ", " ").strip()

    doc = nlp(query)
    sentences = [sent.text.strip() for sent in doc.sents]
    return " ".join([sent for sent in sentences if len(sent) > 0 and sent[-1] == '.']).lower()

def search(query, top_k=10):
    query_embedding = model.encode([clean_query(query)], convert_to_tensor=True, prompt_name="query").cpu().numpy()
    distances, indices = index.search(query_embedding, k=top_k*4)

    already_in = set()
    results = []
    for i in indices[0]:
        id = chunks_ids[i]
        if id in already_in:
            continue

        already_in.add(id)
        results.append(id)
        if len(results) == top_k:
            break

    return df.iloc[results]

In [ ]:
def skewness(query, top_k=10):
    query_embedding = model.encode([clean_query(query)], convert_to_tensor=True, prompt_name="query").cpu().numpy()
    distances, _ = index.search(query_embedding, k=top_k)
    similarities = 1 - distances
    return skew(similarities[0])

## Evaluación preliminar
Dado que no tenemos datos etiquetados, se harán 5 queries para evaluar manualmente la relevancia de los documentos recuperados. Asimismo, se mostrará el query performance prediction para cada query en top10 y top5.

Las consultas serán:
- Monopolio de Sevilla
- Quiero encontrar información sobre la Expedición Malaspina
- Dame información sobre la Ruta Antillana del Cafe
- Buscame artículos que hablen sobre los aymaras del altiplano peruano
- Que datos existen sobre la agricultura prehispánica?

**Resultados**:
- Todos los artículos son relevantes
- El primer documento relevante está en la posición 1
- **Precisión: 1.0**
- $AP(Q1) = \frac{\frac{1}{1}+\frac{2}{2}+...+\frac{10}{10}}{10} = 1.0$

In [50]:
q = rewrite("Monopolio de Sevilla")

print(f"Consulta reformulada: {q}")
print(f"Skewness top10", skewness(q, top_k=10))
print(f"Skewness top5", skewness(q, top_k=5))

search(q, top_k=10)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Consulta reformulada: El monopolio de Sevilla se refiere a un período de tiempo en el que la ciudad de Sevilla, en la región de Andalucía, España, ejerció un dominio absoluto sobre el comercio y la industria en el norte de España, especialmente en la región de Castilla-La Mancha y la región de Extremadura. Este monopolio se estableció en el siglo XV y duró hasta el siglo XVIII.

Durante este período, la ciudad de Sevilla se convirtió en un centro comercial y económico importante, gracias a su posición estratégica en la costa mediterránea y su acceso a rutas comerciales importantes. Sin embargo, el monopolio también llevó a una serie de conflictos y competencias entre Sevilla y otras ciudades y regiones de España, como la ciudad de Córdoba y la región de Castilla.

Uno de los factores que contribuyeron al surgimiento del monopolio fue la expansión de las rutas comerciales marítimas, que conectaban la ciudad de Sevilla con las ciudades de la costa mediterránea y con las ciudades de la re

,Path,Text
902,pack\Revista-de-Indias\2000-El-discutido-monopolio-de-Sevilla.pdf,"así se comprende que los reyes católicos decidan crear en sevilla, en fecha tan temprana como 15..."
545,pack\Anuario-de-Estudios-Americanos\2007-Nobleza-y-fiscalidad-en-la-Ruta-de-las-Indias-el-empori...,del comienzo de la explotación sistemática de las posibilidades expansivas del imperio americano...
1158,pack\Revista-de-Indias\2010-La-institucionalización-de-la-cosmografía-americana-la-Casa-de-la-Co...,los descubrimientos han despertado cierto interés en el ámbito académico internacional. la activ...
762,pack\Anuario-de-Estudios-Americanos\2018-El-comercio-colonial-español-de-la-Carrera-de-Indias-hi...,redes e imperio: aproximación teórica en el caso del comercio atlántico español una revisión que...
510,pack\Anuario-de-Estudios-Americanos\2005-Los-orígenes-de-la-crisis-de-15411543-en-la-política-in...,pone este trabajo es que el desarrollo de las fuerzas económicas derivado directamente de la evo...
546,pack\Anuario-de-Estudios-Americanos\2007-Pinturas-novohispanas-en-España-responsables-finalidad-...,pana situada en nuestro país y los avatares que incidieron en su proceso de llegada. muchas de e...
1296,pack\Revista-de-Indias\2016-La-ropa-de-la-China-desde-Filipinas-hasta-Buenos-Aires-Circulación-c...,pleno entre los historiadores al concebir el contrabando como una de las herramientas más contun...
777,pack\Anuario-de-Estudios-Americanos\2018-Nuevos-tiempos-nuevos-retos-historiográficos-Historia-d...,"introducción la ciudad de sevilla, actual capital de andalucía, tiene una trayectoria americanis..."
945,pack\Revista-de-Indias\2001-Santo-seña-y-ruta-histórica-del-plátano-hasta-Cuba.pdf,"de la isla, permitieron que algunas de sus variedades frutales se convirtieran en objeto de tran..."
1063,pack\Revista-de-Indias\2006-Los-negocios-ultramarinos-de-una-burguesía-cosmopolita-Los-catalanes...,"mejorando los sistemas de navegación, los mecanismos de pago y tejiendo las coaliciones comercia..."


**Resultados**:
- Hay 6 artículos relevantes (2, 6, 8, 10)
- El primer documento relevante está en la posición 1
- **Precisión: 6/10 = 0.6**
- $AP(Q2) = \frac{\frac{1}{1}+\frac{2}{3}+\frac{3}{4}+\frac{4}{5}+\frac{5}{7}+\frac{6}{9}}{6} = 0.76$

In [51]:
q = rewrite("Quiero encontrar información sobre la Expedición Malaspina")

print(f"Consulta reformulada: {q}")
print(f"Skewness top10", skewness(q, top_k=10))
print(f"Skewness top5", skewness(q, top_k=5))

search(q, top_k=10)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Consulta reformulada: La Expedición Malaspina fue un viaje explorador y comercial que se llevó a cabo en la década de 1690 por la empresa de comercio holandesa Van der Donck. Fue liderada por el capitán Juan Sebastián Elcano, quien lideró la expedición hasta su desembarco en la costa de California en 1697. La expedición fue un intento de explorar y establecer una ruta comercial entre Asia y América, y también buscaba encontrar un camino para establecer una colonia en la costa de California.
Skewness top10 0.72301927241953
Skewness top5 0.500208057171255


,Path,Text
558,pack\Anuario-de-Estudios-Americanos\2008-El-gran-mapa-de-la-humanidad-y-las-Bellas-Artes-prehisp...,"xvi y xvii como martín fernández navarrete, reconoció en su relación del viage hecho por las gol..."
665,pack\Anuario-de-Estudios-Americanos\2013-Una-experiencia-reciente-en-el-Pacífico-La-expedición-M...,"numerosas expediciones han sido organizadas y llevadas a cabo por parte de españa, desde las pri..."
652,pack\Anuario-de-Estudios-Americanos\2013-De-la-hidrografía-imperial-a-la-hidrografía-nacional-Re...,"hasta 1835 la exploración hidrográfica del pacífico sur, esto es frente a las costas de chile, f..."
367,pack\Anuario-de-Estudios-Americanos\1997-Tomás-de-Suria-un-dibujante-de-la-expedición-de-Malaspi...,"varios motivos influyeron para que se llevaran a cabo, entre ellos los fines políticos del estad..."
1481,pack\Revista-Española-de-Antropología-Americana\2003-Córdoba-y-Malaspina-antropología-y-política...,el imaginario colectivo admite aún hoy que la guerra contra el indígena que despobló el sur arge...
662,pack\Anuario-de-Estudios-Americanos\2013-Las-rivalidades-hispanoneerlandesas-en-el-Pacífico-y-la...,antecedentes: primeras expediciones en el contexto histórico-político desde el siglo xvi se orga...
1478,pack\Revista-Española-de-Antropología-Americana\2002-Los-saberes-indígenas-y-la-ciencia-de-la-Il...,"usted objetará: ¿para qué sirven las categorías, las variedades y los sistemas? yo le contesto: ..."
1686,pack\Revista-Española-de-Antropología-Americana\2012-Los-pimas-del-padre-Kino.pdf,"de allí cruzar el océano que le llevaría a su destino. aunque su prioridad era misionar en asia,..."
765,pack\Anuario-de-Estudios-Americanos\2018-ElAnuario-de-Estudios-Americanosy-el-despertar-de-la-Hi...,la historia marítima es un campo del saber histórico cuyo estudio científico 1 se comenzó a abor...
1316,pack\Revista-de-Indias\2017-La-presencia-española-en-el-norte-de-Sulawesi-durante-el-siglo-XVII-...,"vino motivada con el objetivo del control de las especias de este archipiélago, el descubrimient..."


**Resultados**:
- Hay 5 artículos relevantes (2, 4, 5, 8 y 9 no lo son)
- El primer documento relevante está en la posición 1
- **Precisión: 5/10 = 0.5**
- $AP(Q3) = \frac{\frac{1}{1}+\frac{2}{3}+\frac{3}{6}+\frac{4}{7}+\frac{5}{10}}{5} = 0.64$

In [53]:
q = rewrite("Dame información sobre la Ruta Antillana del Cafe")

print(f"Consulta reformulada: {q}")
print(f"Skewness top10", skewness(q, top_k=10))
print(f"Skewness top5", skewness(q, top_k=5))

search(q, top_k=10)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Consulta reformulada: La Ruta Antillana del Café fue un tráfico de esclavos que se llevó a cabo en el siglo XVII y XVIII en la Ruta Caribe, que conectaba las Antillas con América. Esta ruta se estableció en el siglo XVI, cuando el comercio de esclavos se expandió por el Caribe y la costa de América. La Ruta Antillana del Café era una de las ramas más importantes de este tráfico, ya que permitía la exportación de esclavos desde las Antillas hasta América.

La ruta se extendía desde las Antillas, particularmente desde Haití, República Dominicana y Cuba, hasta la costa de América, específicamente en Colombia, Venezuela, Ecuador y Perú. Los esclavos transportados a través de la Ruta Antillana del Café eran principalmente de africanos, principalmente de los grupos Bantu y Khoi-Khmer, que se convirtieron en la mayoría de la población esclava en América Latina.

La Ruta Antillana del Café fue un sistema complejo de esclavitud que involucraba a varios actores, incluyendo los traficantes, los
S

,Path,Text
1276,pack\Revista-de-Indias\2015-Los-primeros-tiempos-de-Cuba-en-la-ruta-antillana-del-café.pdf,"uno de estos cultivos, el del café, así como su procesamiento, comercialización y consumo fue de..."
1107,pack\Revista-de-Indias\2008-El-arriero-y-el-transporte-terrestre-en-el-Cono-Sur-Mendoza-17801800...,"siglo xix. como resultado, el comercio bioceánico del cono sur, durante 300 años estuvo en buena..."
1206,pack\Revista-de-Indias\2012-Migración-africana-y-formación-social-en-las-Américas-15002000.pdf,"miento se dirigió a regiones donde la población aborigen era exigua, o había desaparecido como g..."
1416,pack\Revista-de-Indias\2022-El-comercio-interprovincial-de-esclavos-en-el-nororiente-del-Nuevo-R...,introducción los estudios contemporáneos acerca del tráfico de esclavos se han concentrado en in...
849,pack\Anuario-de-Estudios-Americanos\2022-El-capitalismo-de-Génova-y-Burgos-y-la-apertura-de-la-r...,1522. los comienzos de la ruta negrera de santo tomé al caribe la apertura de una ruta negrera d...
1464,pack\Revista-de-Indias\2023-Un-episodio-desconocido-de-la-trata-intercaribeña-de-esclavizados-en...,la de adoptar una perspectiva amplia que ensanche la problemática de estudio queda patente en el...
1146,pack\Revista-de-Indias\2010-Brazos-para-el-azúcar-esclavos-para-vender-Estrategias-de-comerciali...,este trabajo pretende explicar las estrategias comerciales seguidas por los tratantes santiaguer...
510,pack\Anuario-de-Estudios-Americanos\2005-Los-orígenes-de-la-crisis-de-15411543-en-la-política-in...,pone este trabajo es que el desarrollo de las fuerzas económicas derivado directamente de la evo...
875,pack\Anuario-de-Estudios-Americanos\2023-La-participación-de-la-familia-Sánchez-de-Tagle-en-el-t...,introducción la historiografía reciente relacionada con la esclavitud en la monarquía hispánica ...
1187,pack\Revista-de-Indias\2011-Los-Despaigne-en-SaintDomingue-y-Cuba-narrativa-microhistórica-de-un...,"de los integrantes del partido a ejercitar nuestros derechos de cubanos en lo adelante, exigir l..."


**Resultados**:
- Todos los artículos son relevantes
- El primer documento relevante está en la posición 1
- **Precisión: 1.0**
- $AP(Q4) = \frac{\frac{1}{1}+\frac{2}{2}+...+\frac{10}{10}}{10} = 1.0$

In [73]:
q = rewrite("Buscame artículos que hablen sobre los aymaras del altiplano peruano")

print(f"Consulta reformulada: {q}")
print(f"Skewness top10", skewness(q, top_k=10))
print(f"Skewness top5", skewness(q, top_k=5))

search(q, top_k=10)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Consulta reformulada: Artículos que abordan las complejidades de la historia y la cultura de los aymaras del altiplano peruano, como su dinámica social, política y económica, su relación con la colonización española, su evolución en el contexto de la independencia y la formación de la nación peruana, su participación en la lucha por la justicia social y su papel en la construcción de la identidad aymara.
Skewness top10 1.3397564681891203
Skewness top5 0.8596469062068351


,Path,Text
1916,pack\Revista-Española-de-Antropología-Americana\2024-Los-aymaras-del-altiplano-peruano-en-el-bic...,"cómo citar: alanoca-arocutipa, vicente, jorge apaza-ticona, romel percy melgarejo-bolivar y cesa..."
1537,pack\Revista-Española-de-Antropología-Americana\2006-Apxatas-de-difuntos-en-el-altiplano-aymara-...,"en su cautiverio, las almas realizan un trabajo que, semejante al que corresponde a los seres hu..."
0,pack\AIBR\2006-Aymaras-peruanos-y-chilenos-en-los-Andes-ariqueños-Resistencia-y-conflicto-frente...,"catalogada y caricaturizada bajo el rótulo de indómito pueblo araucano. esta imagen, construida ..."
1677,pack\Revista-Española-de-Antropología-Americana\2012-Crianzas-mutuas-El-trato-a-los-animales-des...,1. introducción este trabajo se propone abordar la relación entre las personas y los animales do...
1853,pack\Revista-Española-de-Antropología-Americana\2020-Tiempos-cristianos-y-tiempos-andinos-en-las...,"1. un vaso colonial dedicado a franklin pease, para quien el problema del registro temporal de l..."
1684,pack\Revista-Española-de-Antropología-Americana\2012-Lloren-las-ranas-casen-las-aguas-conténgans...,"dependen la fertilidad de las cosechas y la reproducción de los ganados, y por ende la supervive..."
150,pack\AIBR\2018-De-fantasma-a-vecino-El-Kharisiri-y-la-noción-de-humanidad-en-Los-Andes-aymara-pe...,introducción el presente artículo estudia la figura del kharisiri en relación con las nociones d...
1726,pack\Revista-Española-de-Antropología-Americana\2015-Desde-el-diabólico-mundo-de-los-gentiles-Le...,de potosí y chuquisaca. seguidamente rastrearemos los orígenes del concepto y del término a la l...
1763,pack\Revista-Española-de-Antropología-Americana\2017-Casas-en-las-sociedades-aymaras-norpotosina...,"consenso en la etnohistoria, la etnografía y la arqueología andinas. el modelo, surgido del inte..."
1642,pack\Revista-Española-de-Antropología-Americana\2010-Norte-contra-Sur-Análisis-comparativo-sobre...,"en los andes del sur, en pleno altiplano peruano y boliviano, los maestros rituales aymaras, con..."


**Resultados**:
- Hay 9 documentos relevantes (el 5 no lo es)
- El primer documento relevante está en la posición 1
- **Precisión: 9/10 = 0.9**
- $AP(Q5) = \frac{\frac{1}{1}+\frac{2}{2}+\frac{3}{3}+\frac{4}{4}+\frac{5}{6}+\frac{6}{7}+\frac{7}{8}+\frac{8}{9}+\frac{9}{10}}{9} = 0.93$

In [72]:
q = rewrite("Que datos existen sobre la agricultura prehispánica?")

print(f"Consulta reformulada: {q}")
print(f"Skewness top10", skewness(q, top_k=10))
print(f"Skewness top5", skewness(q, top_k=5))

search(q, top_k=10)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Consulta reformulada: La agricultura prehispánica en América Latina es un tema fascinante que ha sido objeto de estudio y debate entre los historiadores y antropólogos. A continuación, te presento una visión general de los datos existentes sobre la agricultura prehispánica en la región:

* La agricultura prehispánica en América Latina se desarrolló en un período que se extiende desde hace al menos 10.000 a 4.000 años, con algunas evidencias de agricultura en el período precolombino.
* La principal planta alimentaria prehispánica en América Latina era el maíz, que se introdujo en el continente hace alrededor de 7.000 a 10.000 años.
* La agricultura prehispánica en América Latina se caracterizaba por la utilización de técnicas de siembra y cultivo de maíz, trigo, avena, quinoa y otros cereales.
* La agricultura prehispánica en América Latina también se caracterizaba por la producción de alimentos no alimentarios, como frutas, verduras, nueces, semillas y otros productos vegetales
Skewnes

,Path,Text
1743,pack\Revista-Española-de-Antropología-Americana\2015-Variabilidad-en-las-trayectorias-de-adopció...,figura 1: a) noroeste argentino; b) chile central centro-occidente argentino; d) sierras central...
1735,pack\Revista-Española-de-Antropología-Americana\2015-Huertos-chacras-y-sementeras-Plantas-cultiv...,alfarero temprano. 6. los grupos llolleo del período alfarero temprano y la incorporación de nue...
1725,pack\Revista-Española-de-Antropología-Americana\2015-Demografía-humana-e-incorporación-de-cultíg...,"sumario: 1. introducción. 2. centro occidente argentino, registro arqueológico y estrategias hum..."
1723,pack\Revista-Española-de-Antropología-Americana\2015-Boceto-para-un-esquema-domesticación-y-agri...,"presentativos en su caracterización y análisis. lejos de un esquema que se entienda como, o se c..."
37,pack\AIBR\2007-La-combinación-de-sistemas-agrícolas-tradicionales-y-comerciales-el-proceso-de-co...,"how this combination was made, and which were the mechanisms of social and cultural adjustment t..."
1792,pack\Revista-Española-de-Antropología-Americana\2018-El-empleo-de-fotogrametría-mediante-vehícul...,sumario. 1. introducción. la cuenca de méxico y el patrimonio arqueológico en riesgo. 2. las chi...
1724,pack\Revista-Española-de-Antropología-Americana\2015-Cambios-y-permanencias-en-torno-al-proceso-...,"más allá de los matices entre diferentes investigadores, existió un cierto consenso en definir a..."
1736,pack\Revista-Española-de-Antropología-Americana\2015-Interacciones-entre-las-sociedades-y-las-pl...,figura 1: ubicación del área septentrional del río de la plata en el contexto regional. en detal...
1784,pack\Revista-Española-de-Antropología-Americana\2017-Sociedades-heterárquicas-en-el-Ecuador-prei...,1. introducción la presente investigación se centra en la organización política del área cultura...
886,pack\Anuario-de-Estudios-Americanos\2024-De-los-árboles-peces-y-animales-desta-tierra-reconstruc...,introducción este trabajo parte del estudio de dos documentos que integran una unidad: la relaci...


### Resultados generales
$$|Q| = \text{cantidad de consultas} = 5$$
#### MAP@10 (mean average precision)
$$MAP = \frac{1}{|Q|} \sum_{q \space \epsilon \space Q} AP(q) = \frac{1.0+0.76+0.64+1.0+0.93}{5}=0.866 \rightarrow 86.6\%$$
#### MRR@10 (mean reciprocal rank)
$$MRR = \frac{1}{|Q|} \sum_{i=1}^{|Q|} \frac{1}{rank_i} = \frac{1}{5} (\frac{1}{1} + \frac{2}{2} + ... + \frac{5}{5}) = \frac{5}{5} = 1.0 $$